In [7]:
import random
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Adj Close']

def objective_function(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    sharpe_ratio = portfolio_returns.mean() / portfolio_volatility
    return sharpe_ratio


def generate_initial_population(population_size, num_assets):
    population = []
    for _ in range(population_size):
        weights = np.array([random.uniform(0, 1) for _ in range(num_assets)])
        weights /= np.sum(weights)  # Normalize to sum up to 1
        population.append(weights)
    return population


def selection(population, returns, num_parents):
    fitness_scores = [objective_function(weights, returns) for weights in population]
    selected_parents = []

    for _ in range(num_parents):
        # Tournament selection: randomly select individuals and choose the best one
        tournament_size = 5
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_scores[i] for i in tournament_indices]
        winner_index = tournament_indices[tournament_fitness.index(max(tournament_fitness))]
        selected_parents.append(population[winner_index])

    return selected_parents

def crossover(parent1, parent2):
    min_length = min(len(parent1), len(parent2))
    crossover_point = random.randint(1, min_length - 1)
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child


def mutation(individual, mutation_rate):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            # Random mutation: assign a random value between 0 and 1
            mutated_individual[i] = random.uniform(0, 1)

    mutated_individual /= np.sum(mutated_individual)  # Normalize to sum up to 1
    return mutated_individual

def genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents):
    returns = data.pct_change().dropna()
    num_assets = len(data.columns)
    population = generate_initial_population(population_size, num_assets)

    for generation in range(num_generations):
        fitness_scores = [objective_function(weights, returns) for weights in population]
        selected_parents = selection(population, returns, num_parents)

        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            offspring.append(child)

        mutated_offspring = [mutation(individual, mutation_rate) for individual in offspring]

        population = selected_parents + mutated_offspring

    best_individual = max(population, key=lambda x: objective_function(x, returns))
    best_portfolio_allocation = pd.Series(best_individual, index=data.columns)

    return best_portfolio_allocation

# Example usage
tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'META']
start_date = '2010-01-01'
end_date = '2022-12-31'
data = pd.concat([fetch_stock_data(ticker, start_date, end_date) for ticker in tickers], axis=1)
data.columns = tickers

population_size = 100
num_generations = 100
mutation_rate = 0.1
num_parents = 50

best_allocation = genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents)
print(best_allocation)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AAPL     0.273822
GOOGL    0.006706
AMZN     0.177718
MSFT     0.541351
META     0.000403
dtype: float64


In [18]:
import random
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Adj Close']

def objective_function(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    sharpe_ratio = portfolio_returns.mean() / portfolio_volatility
    return sharpe_ratio


def generate_initial_population(population_size, num_assets, total_investment):
    population = []
    for _ in range(population_size):
        weights = np.random.dirichlet(np.ones(num_assets))
        weights *= total_investment  
        population.append(weights)
    return population

def selection(population, returns, num_parents):
    fitness_scores = [objective_function(weights, returns) for weights in population]
    selected_parents = []

    for _ in range(num_parents):
        tournament_size = 10
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_scores[i] for i in tournament_indices]
        winner_index = tournament_indices[tournament_fitness.index(max(tournament_fitness))]
        selected_parents.append(population[winner_index])

    return selected_parents

def crossover(parent1, parent2):
    min_length = min(len(parent1), len(parent2))
    crossover_point = random.randint(1, min_length - 1)
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child


def mutation(individual, mutation_rate, total_investment):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = random.randint(0, total_investment)

    mutated_individual /= np.sum(mutated_individual)
    return mutated_individual


def genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents,total_investment):
    returns = data.pct_change().dropna()
    num_assets = len(data.columns)
    
    population = generate_initial_population(population_size, num_assets, total_investment)

    for generation in range(num_generations):
        fitness_scores = [objective_function(weights, returns) for weights in population]
        selected_parents = selection(population, returns, num_parents)

        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            offspring.append(child)

        mutated_offspring = [mutation(individual, mutation_rate,total_investment) for individual in offspring]

        population = selected_parents + mutated_offspring

    best_individual = max(population, key=lambda x: objective_function(x, returns))
    best_portfolio_allocation = pd.Series(best_individual, index=data.columns)

    return best_portfolio_allocation

In [12]:
tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'META']
start_date = '2010-01-01'
end_date = '2022-12-31'
data = pd.concat([fetch_stock_data(ticker, start_date, end_date) for ticker in tickers], axis=1)
data.columns = tickers

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
import pandas as pd

In [14]:
data1 = pd.read_csv('Data/Stock_History/closing.csv')

In [15]:
data1 = data1.set_index('Date')

In [20]:
population_size = 100
num_generations = 10
mutation_rate = 0.1
num_parents = 50

total_investment = 100000

In [21]:
best_allocation = genetic_algorithm(data1, population_size, num_generations, mutation_rate, num_parents,total_investment)
print(best_allocation)

A       6.724004e-11
AAL     1.254241e-10
AAP     1.693705e-09
AAPL    8.377705e-12
ABBV    1.433329e-11
            ...     
YUM     1.935927e-11
ZBH     1.465640e-11
ZBRA    1.656836e-08
ZION    1.661035e-11
ZTS     6.070594e-11
Length: 503, dtype: float64


In [24]:
best_allocation.values

array([4.42682252e-13, 7.78572500e-11, 1.46005699e-03, 7.24688505e-13,
       2.18381718e-10, 3.45867427e-11, 8.90971727e-13, 8.60157922e-11,
       2.39437778e-10, 4.27516003e-10, 1.74506305e-02, 3.09159118e-02,
       7.10103433e-05, 9.20455617e-13, 9.21076639e-04, 1.56229841e-06,
       8.24970419e-13, 1.64218486e-05, 1.08979009e-03, 9.20621084e-11,
       3.02356182e-10, 4.12426288e-10, 2.60205214e-04, 4.11035544e-05,
       9.94692658e-07, 4.87102945e-11, 3.78749690e-08, 3.45780287e-10,
       7.91890390e-08, 8.41315939e-08, 1.54226401e-11, 1.67140628e-10,
       1.79727242e-10, 1.82292898e-10, 4.93183894e-11, 6.63164615e-11,
       2.29840659e-06, 1.62163383e-10, 3.73665839e-10, 6.37006926e-05,
       2.25524775e-10, 9.01017333e-07, 2.44998967e-10, 2.57420160e-11,
       1.28759789e-13, 1.13680897e-03, 1.63064214e-10, 1.11295780e-10,
       2.57692155e-10, 2.95776275e-06, 5.91804384e-04, 2.72388276e-10,
       3.17475442e-11, 1.24555822e-10, 8.10248672e-08, 9.57240971e-11,
      

In [23]:
import random
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Adj Close']

def objective_function(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    sharpe_ratio = portfolio_returns.mean() / portfolio_volatility
    return sharpe_ratio

def generate_initial_population(population_size, num_assets, total_investment):
    population = []
    for _ in range(population_size):
        weights = np.random.dirichlet(np.ones(num_assets))
        weights *= total_investment  
        population.append(weights)
    return population

def selection(population, returns, num_parents):
    fitness_scores = [objective_function(weights, returns) for weights in population]
    selected_parents = []

    for _ in range(num_parents):
        tournament_size = 10
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_scores[i] for i in tournament_indices]
        winner_index = tournament_indices[tournament_fitness.index(max(tournament_fitness))]
        selected_parents.append(population[winner_index])

    return selected_parents

def crossover(parent1, parent2):
    min_length = min(len(parent1), len(parent2))
    crossover_point = random.randint(1, min_length - 1)
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child

def mutation(individual, mutation_rate, total_investment):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = random.randint(0, total_investment)

    mutated_individual /= np.sum(mutated_individual)
    return mutated_individual

def enforce_two_stock_allocation(individual, selected_indices):
    allocation = np.zeros_like(individual)
    allocation[selected_indices] = individual[selected_indices] / np.sum(individual[selected_indices])
    return allocation

def genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents, total_investment):
    returns = data.pct_change().dropna()
    num_assets = len(data.columns)
    
    population = generate_initial_population(population_size, num_assets, total_investment)

    for generation in range(num_generations):
        fitness_scores = [objective_function(weights, returns) for weights in population]
        selected_parents = selection(population, returns, num_parents)

        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            offspring.append(child)

        mutated_offspring = [mutation(individual, mutation_rate, total_investment) for individual in offspring]

        population = selected_parents + mutated_offspring

    best_individual = max(population, key=lambda x: objective_function(x, returns))
    
    # Enforcing two-stock allocation
    selected_indices = np.argsort(best_individual)[-2:]  # Selecting the indices of the two highest weights
    best_allocation = enforce_two_stock_allocation(best_individual, selected_indices)
    best_portfolio_allocation = pd.Series(best_allocation, index=data.columns)

    return best_portfolio_allocation

tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'META', 'NFLX', 'TSLA', 'NVDA', 'JPM']
start_date = '2010-01-01'
end_date = '2022-12-31'
data = pd.concat([fetch_stock_data(ticker, start_date, end_date) for ticker in tickers], axis=1)
data.columns = tickers

population_size = 100
num_generations = 10
mutation_rate = 0.1
num_parents = 50

total_investment = 100000

best_allocation = genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents, total_investment)
print(best_allocation)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AAPL     0.000000
GOOGL    0.000000
AMZN     0.000000
MSFT     0.000000
META     0.000000
NFLX     0.000000
TSLA     0.000000
NVDA     0.582914
JPM      0.417086
dtype: float64


In [26]:
import random
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']  # Use 'Close' instead of 'Adj Close'

def objective_function(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    sharpe_ratio = portfolio_returns.mean() / portfolio_volatility
    return sharpe_ratio

def generate_initial_population(population_size, num_assets, total_investment):
    population = []
    for _ in range(population_size):
        weights = np.random.dirichlet(np.ones(num_assets))
        weights *= total_investment  
        population.append(weights)
    return population

def selection(population, returns, num_parents):
    fitness_scores = [objective_function(weights, returns) for weights in population]
    selected_parents = []

    for _ in range(num_parents):
        tournament_size = 10
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_scores[i] for i in tournament_indices]
        winner_index = tournament_indices[tournament_fitness.index(max(tournament_fitness))]
        selected_parents.append(population[winner_index])

    return selected_parents

def crossover(parent1, parent2):
    min_length = min(len(parent1), len(parent2))
    crossover_point = random.randint(1, min_length - 1)
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child

def mutation(individual, mutation_rate, total_investment):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = random.randint(0, total_investment)

    mutated_individual /= np.sum(mutated_individual)
    return mutated_individual

def enforce_stock_allocation(individual, selected_indices, remaining_amount):
    allocation = np.zeros_like(individual)
    allocation[selected_indices] = individual[selected_indices] / np.sum(individual[selected_indices])
    remaining_weights = remaining_amount / np.sum(individual[selected_indices])
    allocation[selected_indices] += remaining_weights
    return allocation

def genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents, total_investment, num_stocks):
    returns = data.pct_change().dropna()
    num_assets = len(data.columns)
    
    population = generate_initial_population(population_size, num_assets, total_investment)

    for generation in range(num_generations):
        fitness_scores = [objective_function(weights, returns) for weights in population]
        selected_parents = selection(population, returns, num_parents)

        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            offspring.append(child)

        mutated_offspring = [mutation(individual, mutation_rate, total_investment) for individual in offspring]

        population = selected_parents + mutated_offspring

    best_individual = max(population, key=lambda x: objective_function(x, returns))
    
    # Selecting the indices of the highest weights for the specified number of stocks
    selected_indices = np.argsort(best_individual)[-num_stocks:]

    # Calculating the remaining amount for allocation
    remaining_amount = total_investment - np.sum(best_individual[selected_indices])
    
    # Enforcing stock allocation and remaining amount
    best_allocation = enforce_stock_allocation(best_individual, selected_indices, remaining_amount)
    best_portfolio_allocation = pd.Series(best_allocation, index=data.columns)

    return best_portfolio_allocation

tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'META', 'FB', 'NFLX', 'TSLA', 'NVDA', 'JPM']
start_date = '2010-01-01'
end_date = '2022-12-31'
data = pd.concat([fetch_stock_data(ticker, start_date, end_date) for ticker in tickers], axis=1)
data.columns = tickers

population_size = 100
num_generations = 10
mutation_rate = 0.1
num_parents = 50
total_investment = 100000
num_stocks = 2

best_allocation = genetic_algorithm(data, population_size, num_generations, mutation_rate, num_parents, total_investment, num_stocks)
print(best_allocation)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- FB: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


f:\anaconda\envs\Project\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
f:\anaconda\envs\Project\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: divide by zero encountered in divide
  ret = um.true_divide(
f:\anaconda\envs\Project\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
f:\anaconda\envs\Project\lib\site-packages\pandas\core\frame.py:10222: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
f:\anaconda\envs\Project\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
f:\anaconda\envs\Project\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\Richard Patrick\AppData\Local\Temp\ipykernel_14880\1760878789.py:13:

AAPL     100000.178387
GOOGL         0.000000
AMZN          0.000000
MSFT          0.000000
META          0.000000
FB            0.000000
NFLX          0.000000
TSLA          0.000000
NVDA          0.000000
JPM      100001.178363
dtype: float64
